In [2]:
import torch
from torch import nn
from torch import optim 
from torchvision import datasets, transforms, models
import numpy as np

In [3]:
data_dir = './DATABASE/Final Data/F'

batch_size = 8

test_size = 0.2


transform = transforms.Compose([transforms.Resize(314),
                                transforms.CenterCrop(299),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(10),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                    [0.229, 0.224, 0.225])
                               ])

dataset = datasets.ImageFolder(data_dir, transform=transform)
from torch.utils.data.sampler import SubsetRandomSampler

num_data = len(dataset)
indices = list(range(num_data))
np.random.shuffle(indices)
split = int(np.floor(test_size * num_data))

test_idx = indices[:split]

test_sampler = SubsetRandomSampler(test_idx)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_sampler) # type: ignore

In [4]:
from torchvision.models import Inception_V3_Weights

model = models.inception_v3(weights=Inception_V3_Weights.DEFAULT)
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [5]:
for param in model.parameters():
    param.requires_grad = False
    
fc = nn.Sequential(nn.Linear(2048, 1024),
                          nn.ReLU(),
                          nn.Dropout(0.3),
                          nn.Linear(1024, 512),
                          nn.ReLU(),
                          nn.Dropout(0.3),
                          nn.Linear(512, 2),
                          nn.LogSoftmax(dim=1))

model.fc = fc

In [6]:
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.0001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [7]:
import os
best_model = max(os.listdir('./Models/'), key=lambda x: int(x.split('_')[2].split('.')[0]))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(f'./Models/{best_model}'))
model.to(device)

img, label = next(iter(test_loader))
len(test_loader)

2

In [21]:
cls_len = 2

test_loss = 0.0
accuracy = 0
train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.eval()
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.to(device), target.to(device)
    log_ps = model(data)
    test_loss += criterion(log_ps, target)
                
    ps = torch.exp(log_ps)
    top_p, top_class = ps.topk(1, dim=1)
    equals = top_class == target.view(*top_class.shape)
    accuracy += torch.mean(equals.type(torch.FloatTensor))
    
print("Test loss: {:.3f}.. \t Test Accuracy: {:.3f}".format(test_loss/len(test_loader), accuracy/len(test_loader)))


Test loss: 0.541.. 	 Test Accuracy: 0.875
